In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from matplotlib.image import imread

# Create dataset of image paths in test folder
paths_ds = tf.data.Dataset.list_files(str("./VOC2007_test_subimages/*.jpg"), shuffle=False)
print(paths_ds)

# Helpfer function to extract class/label names from paths
def classnamefromPath(path):
    parts = tf.strings.split(path, os.sep)
    filename = parts[-1]
    label = tf.strings.split(filename, '_')[-1]
    label = tf.strings.split(label, '.')[0]

    return label

# Create list of label/class names
labels = paths_ds.map(classnamefromPath)
labels = np.array(list(labels.as_numpy_iterator()))
labels = [label.decode('utf-8') for label in labels]
label_names = np.unique(labels)
print(label_names)

# Helper fuction to convert string labels to class number
def indexfromClassname(name, labels):
    idx = np.where(labels == name)
    return idx[0][0]

# Create list of class indices
labels_indices = [indexfromClassname(label, label_names) for label in labels]
labels_indices = np.array(labels_indices)
print(labels_indices.shape)

# Encode class indices in one-hot form
target_ds = tf.one_hot(labels_indices, len(label_names), on_value=1.0, off_value=0.0)
target_ds = tf.data.Dataset.from_tensor_slices(target_ds)
print(target_ds)

<TensorSliceDataset shapes: (), types: tf.string>
['aeroplane' 'bicycle' 'bird' 'boat' 'bottle' 'bus' 'car' 'cat' 'chair'
 'cow' 'diningtable' 'dog' 'horse' 'motorbike' 'person' 'pottedplant'
 'sheep' 'sofa' 'train' 'tvmonitor']
(5683,)
<TensorSliceDataset shapes: (20,), types: tf.float32>


In [2]:
# Helper function to read images and pre-process images
# InceptionV3 accepts input of size (299,299,3)
# The input range is (-1,1)
def readImages(path):
    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image)
    image = tf.image.resize_with_pad(image, 299, 299)
    image = keras.applications.inception_v3.preprocess_input(image)

    return image

In [3]:
# print some information to check on images, label and target
path = next(iter(paths_ds))
image = readImages(path)

print("Path: " + str(path))
print("Label: " + str(next(iter(labels))))
print(image.shape)
print(np.max(image))
print(np.min(image))

Path: tf.Tensor(b'.\\VOC2007_test_subimages\\000001_dog.jpg', shape=(), dtype=string)
Label: dog
(299, 299, 3)
1.0
-1.0


In [4]:
# Create dataset of input data, pre-processed images
images_ds = paths_ds.map(readImages)
print(images_ds)

<MapDataset shapes: (299, 299, None), types: tf.float32>


In [5]:
# Create main dataset from input and target datasets
test_dataset = tf.data.Dataset.zip((images_ds, target_ds))
test_dataset = test_dataset.batch(8)
print(test_dataset)

<BatchDataset shapes: ((None, 299, 299, None), (None, 20)), types: (tf.float32, tf.float32)>


In [6]:
# Create pre-trained model
inceptionv3 = keras.applications.inception_v3.InceptionV3(weights='imagenet', input_shape=(299, 299, 3), include_top=False)

# Add global spatial average pooling layer
x = inceptionv3.output
x = keras.layers.GlobalAveragePooling2D()(x)

# Add a fully-connected layer to the raw output of network
x = keras.layers.Dense(512, activation='relu')(x)

# Add a drop-out layer
x = keras.layers.Dropout(rate=0.5)(x)

# Add a logistic layer (softmax) - to predict 10 classes
predictions = keras.layers.Dense(20, activation='softmax')(x)

# Compose the model based on new top-layer
new_inceptionv3 = keras.models.Model(inputs=inceptionv3.input, outputs=predictions)

# First: Train the new top-layer only
# Hence, freeze all layers in pre-trained model
for layer in inceptionv3.layers:
    layer.trainable = False

# Compile model, ready to be trained on new data
new_inceptionv3.compile(optimizer=keras.optimizers.RMSprop(lr=0.001, rho=0.9, decay=0.0001), loss=keras.losses.CategoricalCrossentropy(), metrics=[keras.metrics.CategoricalAccuracy()])

In [7]:
# Evaluate the model
eval = new_inceptionv3.evaluate(test_dataset)
print(eval)

711/711 [==============================] - 328s 462ms/step - loss: 3.2638 - categorical_accuracy: 0.0320
[3.2638261267907507, 0.032025337]
